In [19]:
import torch
import torch.nn as nn
import random
import numpy as np
import Tokenizer as tk
device = "cpu"#torch.device("cuda" if torch.cuda.is_available() else "cpu")
VOCAB_SIZE = 1201

разбивка текста и создание словаря

In [20]:
import os

samples = []
for sample in os.listdir('samples'):
    with open("samples/" + sample, encoding="utf-8") as text:
        samples.append(text.read().lower())


class QAPair:
    def __init__(self, question, answer):
        self.question = question
        self.answer = answer

dataset = []

for sample in samples:
    lines = sample.splitlines()
    questions = lines[::2]
    answers = lines[1::2]
    for q, a in zip(questions, answers):
        dataset.append(QAPair(q, a))


In [21]:
tdg = tk.TokenDictionaryGenerator(vocabulary_size = VOCAB_SIZE-1)
tokens = tdg.generate_tokens(samples)
tokenizer = tk.Tokenizer(tokens)
tokens.save("tokens.json")

создание модели 
предложение -> hidden
последние слово + hidden -> слово(1)...слово(n)

In [22]:
class RnnTextGen(nn.Module):

    def __init__(self,input_size,inp_lstm_size,hid_size,n_layers,out_size,dropout=0.2) -> None:
        super(RnnTextGen,self).__init__()
        self.input_size = input_size
        self.n_layers = n_layers
        self.hidden_size=hid_size
        self.Encoder = nn.Embedding(input_size,inp_lstm_size)
        self.lstm = nn.LSTM(inp_lstm_size,hid_size,n_layers)
        self.dropout = nn.Dropout(dropout)
        self.l1 = nn.Linear(hid_size,out_size)
        
    def forward(self,x,hidden=None):
        x = self.Encoder(x)
        x,hidden = self.lstm(x)
        x = self.dropout(x)
        x = self.l1(x)
        return x,hidden
    
    def init_hidden(self,batch_size=1):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device),
               torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device))

In [23]:
model=RnnTextGen(VOCAB_SIZE,1000,500,2,VOCAB_SIZE).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, amsgrad=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    patience=5, 
    verbose=True, 
    factor=0.5
)

In [24]:
def evaluate(model:RnnTextGen,text:str,prediction_lim:int=15):
    text_idx = torch.LongTensor(list(tokenizer.tokenize(text))).to(device)
    hidden = model.init_hidden()
    inp = text_idx
    predicted_text=""
    for i in range(prediction_lim):
        next_w , hidden = model(inp.view(-1,1).to(device),hidden)
        inp = torch.cat([inp,next_w[-1].argmax().view(-1)])
        if next_w[-1].argmax() == torch.LongTensor([VOCAB_SIZE-1]).to(device):
            break
        word = tokens.decode(int(next_w[-1].argmax()))
        predicted_text +=word
    return predicted_text

In [25]:
def get_batch(dataset:list):
    for qa in dataset:
        question_idx = list(tokenizer.tokenize(qa.question))
        target = list(tokenizer.tokenize(qa.answer))+[tokens.count()]
        test = question_idx+target[:-1]

        target =torch.LongTensor(target).to(device)
        test = torch.LongTensor(test).to(device)
        yield target,test

In [26]:
def train(epoches:int,model:RnnTextGen,batch_size:int)->None:
    """epoches - number of epoches through all dataset
    model - model required to teach
    batch_size - n/a"""
    loss_avg =[]
    for epoch in range(epoches):
        for target,train in get_batch(dataset):
            model.train()

            hidden = model.init_hidden(batch_size)

            output,hidden = model(train,hidden)
            target_len = len(target)
            loss = criterion(output[-target_len:],target)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            loss_avg.append(loss.item())
            if len(loss_avg) >= 50:
                mean_loss = np.mean(loss_avg)
                print(f'Loss: {mean_loss}')
                scheduler.step(mean_loss)
                model.eval()
                question = random.choice(dataset).question
                answer = evaluate(model,question)
                print(f"Question: {question} \n Answer: {answer}")
                loss_avg = []

обучение модели

In [27]:
train(30, model, 1)

Loss: 6.481576414108276
Question: но как мы можем определить, что правильно, а что нет? 
 Answer: 
Loss: 5.835963954925537
Question: куда ты собираешься? 
 Answer: мой любимый . а ты?
Loss: 5.770610451698303
Question: софия как ты считаешь как можно получить практический опыт? 
 Answer: 
Loss: 4.908107271194458
Question: и как мы можем достичь справедливости в обществе? 
 Answer: я думаю, что наш выбзульссопределены.
Loss: 4.598390336036682
Question: как важен практический опыт в образовании? 
 Answer: мой любимый зверь - й . а у тебя?
Loss: 4.531934795379638
Question: как называется животное, которое может менять цвет? 
 Answer: ваный помочь человеку
Loss: 3.6089592637121677
Question: как важен практический опыт в образовании? 
 Answer: я думаю, что человек может влиять на свою жизнь, но некоторые
Loss: 2.7507587997615337
Question: что делаешь? 
 Answer: я читаю  - футбол. а ты?
Loss: 2.8043147083371878
Question: ты лох  
 Answer: я устал тратить на тебя время
Loss: 2.3568826164864003

KeyboardInterrupt: 

In [29]:
quest = input().lower()
evaluate(model,quest,35)

'да так ничего'

In [ ]:
torch.save(model,"data.pkl")

In [ ]:
model = torch.load("data.pkl").to(device)